In [2]:
from utils.util import mkdir,seed_all
from omegaconf import OmegaConf
from cprint import *
from datasets.shape_net import ShapeNet
import torch
from models.Transform2D import Transform2D
from tqdm.notebook import tqdm
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import os
import numpy as np
import datetime
%load_ext autoreload
%load_ext tensorboard
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Setup Expirement Folder

In [3]:
import time
seed_all(111)
today = time.strftime("%Y-%m-%d")
config = OmegaConf.load("./configs/global_configs.yaml")
cprint.ok(config)
description = "Testing training script" # Describe Experiment params here
logs_dir = config["logs_dir"]
mkdir(logs_dir)
experiment_dir = f"{logs_dir}/{config['name']}"
mkdir(experiment_dir)
loss_log_title = "Loss Log" + today 

with open(f"{experiment_dir}/description.txt", "w") as file1:
    file1.write(description)
    
with open(f"{experiment_dir}/configs.txt", "w") as file1:
    file1.write(str(config))

with open(f"{experiment_dir}/loss_log.txt", "w") as file1:
    file1.write(loss_log_title)
    file1.write("\n")


mkdir(f"{experiment_dir}/checkpoints")
mkdir(f"{experiment_dir}/tb")
mkdir(f"{experiment_dir}/visuals")

{'logs_dir': 'logs', 'is_train': True, 'name': 'trainingV8', 'device': 'cuda:0', 'batch_size': 8, 'n_epochs': 100, 'print_every': 20, 'validate_every': 30, 'save_every': 50, 'save_every_nepochs': 10, 'model': {'lr': 0.0001, 'criterion': 'BCE', 'pos_weight': 1.3, 'encoder': {'patch_size': 13, 'sequence_length': 100, 'embedding_dim': 768, 'patch_padding': 3}, 'transformer_encoder': {'d_model': 768, 'nhead': 12, 'num_layers': 12}, 'transformer_decoder': {'d_model': 768, 'nhead': 12, 'num_layers': 8, 'num_pos_embeddings': 64}}}
- logs directory found


- Creating new directory logs/trainingV8
- Creating new directory logs/trainingV8/checkpoints
- Creating new directory logs/trainingV8/tb
- Creating new directory logs/trainingV8/visuals


## Load Dataset & Dataloaders
This uses a random split for train/validation/test - might need to look into paper if they have a pre-defined split

In [4]:
dataset = ShapeNet(cat="airplane",is_overfit=True) #Change overfit param here & cat here
print('length: ', len(dataset))
train_ds, valid_ds, test_ds = torch.utils.data.random_split(
    dataset, [56, 8, 0])

train_dataloader = torch.utils.data.DataLoader(
        train_ds,   # Datasets return data one sample at a time; Dataloaders use them and aggregate samples into batches
        batch_size=config['batch_size'],   # The size of batches is defined here
        shuffle=True,    # Shuffling the order of samples is useful during training to prevent that the network learns to depend on the order of the input data
        num_workers=4,   # Data is usually loaded in parallel by num_workers
        pin_memory=True,  # This is an implementation detail to speed up data uploading to the GPU
        # worker_init_fn=train_dataset.worker_init_fn  TODO: Uncomment this line if you are using shapenet_zip on Google Colab
    )

validation_dataloader = torch.utils.data.DataLoader(
        train_ds,   # Datasets return data one sample at a time; Dataloaders use them and aggregate samples into batches
        batch_size=config['batch_size'],   # The size of batches is defined here
        shuffle=True,    # Shuffling the order of samples is useful during training to prevent that the network learns to depend on the order of the input data
        num_workers=4,   # Data is usually loaded in parallel by num_workers
        pin_memory=True,  # This is an implementation detail to speed up data uploading to the GPU
        # worker_init_fn=train_dataset.worker_init_fn  TODO: Uncomment this line if you are using shapenet_zip on Google Colab
    )


length:  64


## Loading Model

In [5]:
model = Transform2D()
# Declare device
device = torch.device('cpu')
if torch.cuda.is_available() and config['device'].startswith('cuda'):
    device = torch.device(config['device'])
    cprint.ok('Using device:', config['device'])
else:
    cprint.warn('Using CPU')

model.to(device)


Using device: cuda:0


Transform2D(
  (patch_encoder): PatchEncoder(
    (pos_embedding): Embedding(100, 768)
    (patch_embed): PatchEmbed(
      (proj): Conv2d(4, 768, kernel_size=(13, 13), stride=(13, 13))
      (norm): Identity()
    )
  )
  (transformer_encoder): TransformerEncoder(
    (net): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (linear1): Linear(in_features=768, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=768, bias=True)
          (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): Transf

## Training

Checklist:
- Add tensorboard and make sure it logs to the `${experiment_dir}/tb` folder
- Visualize some reconstructions on validation set and make sure it logs to the `${expirement_dir}/visuals` folders

In [6]:
train_loss_running = 0.
best_loss_val = np.inf
model.train()

tb_dir = f"{experiment_dir}/tb"
writer = SummaryWriter(log_dir=tb_dir)
model_checkpoint_path = f"{experiment_dir}/checkpoints"
loss_log_name = f"{experiment_dir}/loss_log.txt"
last_loss = 0.
def train_one_epoch(epoch, writer):
     global best_loss_val
     global last_loss   
     train_loss_running = 0.
     iteration_count = 0
     for batch_idx, batch in tqdm(enumerate(train_dataloader)):
         ShapeNet.move_batch_to_device(batch, device)
         model.step(batch)
         loss = model.get_loss()["loss"]
         train_loss_running += loss
         iteration = epoch * len(train_dataloader) + batch_idx   
         iteration_count += 1
         if iteration % config['print_every'] == (config['print_every'] - 1):
            avg_train_loss = train_loss_running / iteration_count
            message = '(epoch: %d, iters: %d, loss: %.6f)' % (epoch, iteration, loss.item())
            with open(loss_log_name, "a") as log_file:
                log_file.write('%s\n' % message)
            print(loss)
            cprint.warn(f'[{epoch:03d}/{batch_idx:05d}] train_loss: {avg_train_loss:.6f}')
            writer.add_scalar("Loss/Train", avg_train_loss, iteration)
            last_loss = avg_train_loss
            train_loss_running = 0.
            iteration_count = 0
         
         if iteration % config['save_every'] == (config['save_every'] - 1):
            model.save(model_checkpoint_path, "latest")
            
        
         if iteration % config['validate_every'] == (config['validate_every'] - 1):
            cprint.ok("Running Validation")
            model.eval()
            loss_val = 0.
            output = []
            index_batch = 0
            for batch_val in validation_dataloader:
                ShapeNet.move_batch_to_device(batch_val, device)
                with torch.no_grad():
                    output = model.inference(batch_val)
                    loss_val +=  model.get_loss()["loss"]
                    index_batch += 1
            avg_loss_val = loss_val / (index_batch)
            
            if avg_loss_val < best_loss_val:
                model.save(model_checkpoint_path, "best")
                best_loss_val = avg_loss_val
            
            cprint.warn(f'[{epoch:03d}/{batch_idx:05d}] val_loss: {avg_loss_val:.6f} | best_loss_val: {best_loss_val:.6f}')
            writer.add_scalar("Validation Loss", loss_val, iteration)
            #import pdb;pdb.set_trace();
            writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : last_loss, 'Validation' : avg_loss_val },
                     iteration)
            writer.flush()
      
            return last_loss

     




for epoch in tqdm(range(config['n_epochs'])):
    avg_loss = train_one_epoch(epoch, writer) 
    if(epoch % config["save_every_nepochs"]==0):
        model.save(model_checkpoint_path, epoch)
    model.update_lr()
    writer.close()
            
#cprint.ok(f"Visualizations saved to: {experiment_dir}/visuals")



  0%|          | 0/100 [00:00<?, ?it/s]

0it [00:00, ?it/s]

logs/trainingV8/checkpoints/epoch-0.ckpt created


0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

Running Validation
logs/trainingV8/checkpoints/epoch-best.ckpt created


[098/00003] val_loss: 0.059873 | best_loss_val: 0.059873


0it [00:00, ?it/s]

tensor(0.0655, device='cuda:0')
logs/trainingV8/checkpoints/epoch-latest.ckpt created


[099/00006] train_loss: 0.060575
